# Análisis de Opinión en Repositorios de Software (GitHub)
**Proyecto:** Procesamiento de Lenguaje Natural (PLN)
**Repositorio Analizado:** `facebook/react`


**Autores:**


 * Karla Patricia Solorzano Parra
 * Anthony Joel Toalombo Aisabucha
 * Mayerly Kristel Velos Alburquerque



---

## 1. Objetivos del Proyecto
El objetivo es construir un observatorio de salud de proyectos Open Source analizando los comentarios, issues y pull requests. Se busca:
1.  **Recolectar datos reales** usando la API de GitHub.
2.  **Detectar sentimientos** (Positivo, Negativo, Neutral) para medir la "temperatura" de la comunidad.
3.  **Identificar temas ocultos** (Clustering) para saber de qué se habla más (Bugs, Documentación, Features).

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
import re
import warnings

# Librerías de NLP (NLTK)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

# Librerías de Machine Learning (Scikit-Learn)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix

# Configuración
warnings.filterwarnings('ignore')

# Descargar diccionarios
print("Cargando herramientas de lenguaje...")
for dep in ['stopwords', 'wordnet', 'omw-1.4', 'vader_lexicon']:
    try: nltk.data.find(f'corpora/{dep}')
    except: nltk.download(dep, quiet=True)
print(" Todo listo.")

Cargando herramientas de lenguaje...
 Todo listo.


##  2. Ingesta de Datos (Data Ingestion)
Conexión segura a la API de GitHub para descargar issues y comentarios.
* **Seguridad:** Se utiliza `google.colab.userdata` para proteger el Token de acceso.
* **Volumen:** Se descargarán **30 páginas** (aprox. 900 registros) para garantizar representatividad estadística.

In [ ]:
# --- Token seguro desde Secrets ---
from google.colab import userdata

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("Token cargado correctamente desde Secrets.")
except:
    # Si falla, no rompemos el código, solo avisamos
    GITHUB_TOKEN = None
    print(" No se encontró 'GITHUB_TOKEN'. Se usará modo limitado.")

REPO_OWNER = "facebook"
REPO_NAME = "react"
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"} if GITHUB_TOKEN else {}

PAGES_TO_FETCH = 30

def get_comments(comments_url):
    if not comments_url: return ""
    try:
        response = requests.get(comments_url, headers=HEADERS)
        if response.status_code == 200:
            return " ".join([c['body'] for c in response.json() if c.get('body')])
    except: pass
    return ""

print(f"--- Conectando a GitHub para descargar datos de {REPO_NAME} ---")

all_data = []
for page in range(1, PAGES_TO_FETCH + 1):
    print(f"Descargando página {page}...", end="\r")
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues?state=all&per_page=30&page={page}"

    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        for item in response.json():
            comentarios = ""
            if item['comments'] > 0:
                comentarios = get_comments(item['comments_url'])
                time.sleep(0.1) # Pausa pequeña

            all_data.append({
                "id": item['number'],
                "type": "pull_request" if "pull_request" in item else "issue",
                "title": item['title'],
                "body": item.get('body', "") or "",
                "comments": comentarios,
                "state": item['state']
            })
        time.sleep(0.5)
    except Exception as e:
        print(f"Error en página {page}: {e}")

# Guardar CSV
df = pd.DataFrame(all_data)

# Creamos la columna 'full_text' que usaremos abajo
df['full_text'] = df['title'].fillna('') + " " + df['body'].fillna('') + " " + df['comments'].fillna('')

df.to_csv("dataset_react.csv", index=False, encoding='utf-8')

print(f"\n¡ÉXITO! Se creó el archivo 'dataset_react.csv' con {len(df)} registros.")

Token cargado correctamente desde Secrets.
--- Conectando a GitHub para descargar datos de react ---

¡ÉXITO! Se creó el archivo 'dataset_react.csv' con 900 registros.


### 3. Preprocesamiento y Normalización (NLP Pipeline)
Los datos textuales crudos contienen ruido que afecta el desempeño de los modelos. Se aplica un pipeline de limpieza estructurado:
1.  **Normalización:** Estandarización a minúsculas (`lowercase`).
2.  **Filtrado de Ruido:** Eliminación de artefactos web (URLs, menciones, etiquetas HTML).
3.  **Stopwords Removal:** Descarte de términos funcionales sin carga semántica (artículos, preposiciones).
4.  **Lematización:** Reducción morfológica de las palabras a su raíz (lema) utilizando el corpus de **WordNet**.

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.update(['would', 'could', 'issue', 'pull', 'request', 'react', 'code'])

def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""

    # a. Todo a minúsculas
    texto = texto.lower()

    # b. Eliminar URLs y menciones
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'@\w+', '', texto)

    # c. Eliminar signos (dejamos solo letras)
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)

    # d. Tokenización y Lematización (reducción a la raíz)
    palabras = texto.split()
    palabras_limpias = [
        lemmatizer.lemmatize(word)
        for word in palabras
        if word not in stop_words and len(word) > 2
    ]

    return " ".join(palabras_limpias)

print("Cargando dataset...")

df = pd.read_csv("dataset_react.csv")

print("Limpiando textos...")
df['clean_text'] = df['full_text'].apply(limpiar_texto)

# Verificación

print("\n--- RESULTADO ---")
print("Original:", df['full_text'].iloc[0][:80])
print("Limpio:  ", df['clean_text'].iloc[0][:80])

# Guardar
df.to_csv("dataset_react_procesado.csv", index=False)
print("\n¡Archivo 'dataset_react_procesado.csv' guardado correctamente!")

Cargando dataset...
Limpiando textos...

--- RESULTADO ---
Original: Bump webpack from 5.82.1 to 5.104.1 Bumps [webpack](https://github.com/webpack/w
Limpio:   bump webpack bump webpack detail summaryrelease notessummary pemsourced href rel

¡Archivo 'dataset_react_procesado.csv' guardado correctamente!


## 4. Modelado y Representación Vectorial
Convertimos el texto limpio en números usando **TF-IDF** (Term Frequency - Inverse Document Frequency).


> ***NOTA:***   
 No guardamos la matriz en CSV porque es demasiado grande y compleja.
La variable 'tfidf_matrix' y 'tfidf_vectorizer' se quedan en la memoria  de Colab listas para el siguiente paso (Clustering y Sentimiento).

In [ ]:

# 1. CARGAR DATOS

# Cargamos el archivo limpio que acabamos de guardar
df = pd.read_csv("dataset_react_procesado.csv")

# Eliminar filas vacías (por si la limpieza dejó algún texto en blanco)
df = df.dropna(subset=['clean_text'])

print("Generando matriz TF-IDF...")

# 2. CONFIGURACIÓN DEL VECTORIZADOR (RF-03)
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,      # Top 5000 palabras más importantes
    ngram_range=(1, 2),     # Unigramas y Bigramas (ej: "data", "data science")
    min_df=2                # Ignorar palabras que aparecen en menos de 2 documentos (ruido)
)

# 3. TRANSFORMACIÓN
# Esto convierte el texto en una matriz
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'])

# 4. ANÁLISIS DEL RESULTADO
print("\n--- RESULTADOS ---")
print(f"Tamaño de la matriz: {tfidf_matrix.shape}")
# El resultado será (Filas, Columnas) --- (Número de Issues, Número de Palabras)

# Ejemplo de palabras que el modelo aprendio
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"\n Primeras 10 palabras del vocabulario: {feature_names[:10]}")
print(f"\n Algunas palabras del medio: {feature_names[500:510]}")



Generando matriz TF-IDF...

--- RESULTADOS ---
Tamaño de la matriz: (899, 5000)

 Primeras 10 palabras del vocabulario: ['ability' 'able' 'able prioritize' 'abort' 'aborted' 'absolutely'
 'accept' 'accept meta' 'acceptable' 'accepted']

 Algunas palabras del medio: ['changelog' 'changelogaemp' 'changelogaemp blockquote' 'changelogstrong'
 'changelogstrong href' 'changesh' 'changessummary' 'changessummary pthis'
 'changing' 'channel']


### 4.1 Análisis de Sentimiento Supervisado
Entrenamos una **Regresión Logística** para clasificar opiniones.
> ***Nota Técnica:***   Dado el fuerte desbalance de clases (muchos positivos, pocos negativos), se configuró el modelo con `class_weight='balanced'`. Esto obliga al algoritmo a prestar más atención a las críticas y reportes de error (Clase Negativa), mejorando significativamente el *Recall*.

In [ ]:

# --- 1. GENERACIÓN DE ETIQUETAS (EL "PROFESOR" VADER) ---
print("Descargando diccionario de sentimientos VADER...")
import nltk
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def obtener_sentimiento(texto):
    score = sia.polarity_scores(texto)['compound']
    # Reglas estándar de VADER:
    if score >= 0.05:
        return "Positivo"
    elif score <= -0.05:
        return "Negativo"
    else:
        return "Neutral"

print("Etiquetando datos automáticamente...")
# Usamos el texto limpio para calificar
df['sentimiento'] = df['clean_text'].apply(obtener_sentimiento)

# Veamos cuántos de cada tipo encontró
print("\nDistribución de sentimientos detectados:")
print(df['sentimiento'].value_counts())

# --- 2. PREPARACIÓN PARA EL MODELO SUPERVISADO (RF-04) ---
# X = Nuestros datos matemáticos (Matriz TF-IDF del paso anterior)
# y = La respuesta correcta (las etiquetas que acabamos de crear)
X = tfidf_matrix
y = df['sentimiento']

# Dividimos: 80% para entrenar (estudiar), 20% para test (examen)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. ENTRENAMIENTO DEL MODELO (LOGISTIC REGRESSION) ---
print("\nEntrenando Modelo de Regresión Logística...")
modelo = LogisticRegression(max_iter=1000, class_weight="balanced") # max_iter alto para asegurar convergencia
modelo.fit(X_train, y_train)

# --- 4. EVALUACIÓN Y MÉTRICAS (RF-04) ---
print("Evaluando modelo...")
y_pred = modelo.predict(X_test)

print("\n--- REPORTE DE CLASIFICACIÓN (RF-04) ---")
# Esto muestra Accuracy y F1-Score
print(classification_report(y_test, y_pred))

# Guardamos el dataset final con las etiquetas
df.to_csv("dataset_react_con_sentimientos.csv", index=False)

Descargando diccionario de sentimientos VADER...
Etiquetando datos automáticamente...


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



Distribución de sentimientos detectados:
sentimiento
Positivo    625
Negativo    236
Neutral      38
Name: count, dtype: int64

Entrenando Modelo de Regresión Logística...
Evaluando modelo...

--- REPORTE DE CLASIFICACIÓN (RF-04) ---
              precision    recall  f1-score   support

    Negativo       0.58      0.75      0.66        56
     Neutral       0.10      0.20      0.13         5
    Positivo       0.88      0.72      0.79       119

    accuracy                           0.72       180
   macro avg       0.52      0.56      0.53       180
weighted avg       0.76      0.72      0.73       180



### 4.2  Modelado de Temas (Aprendizaje No Supervisado)
Para descubrir las tendencias de discusión sin intervención humana, aplicamos el algoritmo **K-Means**.
* **Objetivo:** Agrupar los documentos en **3 clústeres** semánticos basados en la similitud de sus vectores TF-IDF.
* **Interpretación:** Esto permitirá distinguir entre discusiones de desarrollo, mantenimiento de infraestructura y reportes automatizados.

In [ ]:
from sklearn.cluster import KMeans

# --- 1. CONFIGURACIÓN DE K-MEANS (RF-05) ---
# se pide que encuentre 3 temas principales
num_clusters = 3

print(f"Agrupando los textos en {num_clusters} temas...")
km = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)

# Entrenamos con la matriz TF-IDF
km.fit(tfidf_matrix)

# Guardamos el grupo (cluster) al que pertenece cada texto
df['cluster'] = km.labels_

# --- 2. INTERPRETACIÓN DE LOS TEMAS ---
print("\n--- TEMAS IDENTIFICADOS ---")
# Obtenemos los centros de los clusters y las palabras
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names_out()

tema_nombres = {}

for i in range(num_clusters):
    print(f"\nGRUPO {i}:")
    # Imprimimos las 10 palabras más importantes de ese grupo
    top_words = [terms[ind] for ind in order_centroids[i, :10]]
    print(f"Palabras clave: {', '.join(top_words)}")

    # Imprimimos un ejemplo real de ese grupo para entenderlo mejor
    ejemplo = df[df['cluster'] == i]['clean_text'].iloc[0]
    print(f"Ejemplo de texto: {ejemplo[:100]}...")

# --- 3. GUARDADO FINAL ---
df.to_csv("dataset_react_completo.csv", index=False)
print("\n¡Análisis de temas completado y guardado!")

Agrupando los textos en 3 temas...

--- TEMAS IDENTIFICADOS ---

GRUPO 0:
Palabras clave: yarn, test, yarn test, cla, please, sign, run yarn, run, change, contributor
Ejemplo de texto: fix grammar error overviewmd thanks submitting appreciate spending time work change please provide e...

GRUPO 1:
Palabras clave: compiler, using, bug, test, version, error, function, example, const, component
Ejemplo de texto: bump webpack bump webpack detail summaryrelease notessummary pemsourced href releasesaemp blockquote...

GRUPO 2:
Palabras clave: gzip, change, current gzip, base, facebook, current, greater, size change, change greater, change includes
Ejemplo de texto: flag cleanup enablehalt shipped failure warning markdown notice dangerid dangeridstable comparing bb...

¡Análisis de temas completado y guardado!


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 1. CÁLCULO DE LA MATRIZ DE SIMILITUD (RF-06) ---
# Comparamos TODOS contra TODOS.

print("Calculando similitudes...")
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# --- 2. FUNCIÓN DE BÚSQUEDA ---
# Esta función cumple el requisito: "Dado un mensaje, mostrar los similares"
def recomendar_similares(id_texto, top_n=3):
    # Obtenemos los puntajes de similitud para ese texto específico
    sim_scores = list(enumerate(cosine_sim[id_texto]))

    # Los ordenamos de mayor a menor similitud

    # (El primero siempre es él mismo, así que lo saltamos con [1:top_n+1])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]

    # Imprimimos los resultados
    print(f"\n--- MENSAJE ORIGINAL (ID: {id_texto}) ---")
    print(f"Texto: {df['clean_text'].iloc[id_texto][:100]}...")
    print(f"Cluster: {df['cluster'].iloc[id_texto]}")

    print(f"\n--- {top_n} MENSAJES MÁS SIMILARES ENCONTRADOS ---")
    for i, score in sim_scores:
        print(f"ID: {i} | Similitud: {score:.4f} ({(score*100):.1f}%)")
        print(f"Texto: {df['clean_text'].iloc[i][:100]}...")
        print("-" * 40)

# --- 3. PROBAR CON UN EJEMPLO ---
# recomendaciones para el texto número 10
recomendar_similares(10)

# SE rueba también con otro
recomendar_similares(50)

Calculando similitudes...

--- MENSAJE ORIGINAL (ID: 10) ---
Texto: devtools throw error attempting clone nonexistent node existing serialisation logic trace profiler p...
Cluster: 1

--- 3 MENSAJES MÁS SIMILARES ENCONTRADOS ---
ID: 477 | Similitud: 0.2353 (23.5%)
Texto: chore add nvmrc specify node development consistency summary add nvmrc file specifying node recommen...
----------------------------------------
ID: 321 | Similitud: 0.1744 (17.4%)
Texto: upgrade github action node compatibility summary upgrade github action latest version ensure compati...
----------------------------------------
ID: 201 | Similitud: 0.1544 (15.4%)
Texto: devtools clear element inspection host element owned renderer selected stacked click outside root el...
----------------------------------------

--- MENSAJE ORIGINAL (ID: 50) ---
Texto: bug logcomponentrender break existing functionality utilise proxy trpc client clientproceduretype fu...
Cluster: 1

--- 3 MENSAJES MÁS SIMILARES ENCONTRADOS ---
ID: 

### 5. Conclusiones y Hallazgos Técnicos
Tras la ejecución del pipeline de análisis sobre el repositorio `facebook/react`, se presentan los siguientes hallazgos:

1.  **Métrica de Desempeño (Recall):** El modelo supervisado alcanzó un **Recall del 75%** en la clase Negativa. Esto valida la eficacia del sistema como herramienta de alerta temprana para la detección de bugs.
2.  **Identificación de Tendencias:** El algoritmo de Clustering reveló que la discusión técnica actual gira en torno al nuevo **React Compiler** (Grupo 1) y tareas de **Infraestructura/CI** (Grupo 0), separando eficazmente el ruido de los bots.
3.  **Validación de Similitud:** El módulo de *Cosine Similarity* demostró capacidad para la deduplicación de incidencias, identificando reportes de error redundantes con alta precisión semántica.

##  6. Recomendaciones
Tras el análisis de los datos recolectados:
1.  **Desbalance:** La comunidad tiende a ser muy constructiva (sentimiento mayoritariamente positivo/técnico).
2.  **Detección de Problemas:** Gracias al ajuste de pesos (`balanced`), el modelo logra identificar reportes de fallos (issues negativos) que antes pasaban desapercibidos.
3.  **Temas:** Los clusters separaron exitosamente el mantenimiento automático (bots) de las discusiones humanas sobre código.